In [1]:
import pandas as pd
import numpy as np
import pickle
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
import string
import re
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as VS
# from textstat.textstat import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn import cross_validation
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.models import model_from_json
from keras.optimizers import Adam

/home/bhargav/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
embeddings_index = dict()
f = open('../glove.6B/glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [3]:
df = pd.read_csv("../Proper Experiments/Parsed Datasets/Waseem_Hovy.csv")
df.text=df.text.astype(str)
df.head(5)

,Unnamed: 0,text,class
0,0,mralmubarak yes tried murder way heaven total...,1
1,1,nigzzzk command fear allah quran ritual grove...,1
2,2,ib alkisai islamic documentation based belief...,1
3,3,benfrancisallen : good weeks isis new front o...,0
4,4,self described radical feminist makes death th...,1


In [4]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
vocabulary_size=len(tokenizer.word_index) + 1
print(vocabulary_size)

embedding_matrix = np.zeros((vocabulary_size, 300))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

23678


In [5]:
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [6]:
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    break

In [7]:
hidden_size=[75,100,125]
learning_rate=[0.0001,0.001,0.01]
score=[]
accuracy=[]

for i in learning_rate:
    sco=[]
    acc=[]
    for j in hidden_size:
        
        model_glove = Sequential()
        model_glove.add(Embedding(vocabulary_size, 300, input_length=80, weights=[embedding_matrix], trainable=False))
        model_glove.add(LSTM(300))
        model_glove.add(Dense(j, activation='relu'))
        model_glove.add(Dense(1, activation='sigmoid'))
        model_glove.compile(loss='binary_crossentropy', optimizer=Adam(lr=i), metrics=['accuracy'])
        
        model_glove.fit(Xtrain, np.array(ytrain), epochs = 3)

        pred=model_glove.predict_classes(X_val)
        
        acc.append(accuracy_score(y_val, pred))
        sco.append(f1_score(y_val, pred, average='macro'))
    score.append(sco)
    accuracy.append(acc)

Epoch 1/3
11367/11367 [==============================] - 65s 6ms/step - loss: 0.5045 - acc: 0.7755
Epoch 2/3
11367/11367 [==============================] - 59s 5ms/step - loss: 0.4383 - acc: 0.8070
Epoch 3/3
11367/11367 [==============================] - 59s 5ms/step - loss: 0.4168 - acc: 0.8158
Epoch 1/3
11367/11367 [==============================] - 60s 5ms/step - loss: 0.4992 - acc: 0.7757
Epoch 2/3
11367/11367 [==============================] - 60s 5ms/step - loss: 0.4350 - acc: 0.8097
Epoch 3/3
11367/11367 [==============================] - 59s 5ms/step - loss: 0.4133 - acc: 0.8217
Epoch 1/3
11367/11367 [==============================] - 60s 5ms/step - loss: 0.5104 - acc: 0.7621
Epoch 2/3
11367/11367 [==============================] - 59s 5ms/step - loss: 0.4365 - acc: 0.8043
Epoch 3/3
11367/11367 [==============================] - 59s 5ms/step - loss: 0.4176 - acc: 0.8180
Epoch 1/3
11367/11367 [==============================] - 59s 5ms/step - loss: 0.4665 - acc: 0.7915
Epoch 2/3


In [8]:
print(score)

[[0.7818851562649886, 0.7645268813303829, 0.7642384325696367], [0.7825965137953876, 0.7927221482210767, 0.7885996997413354], [0.7869854268042262, 0.7718277006855823, 0.7608200886889411]]


In [9]:
print(accuracy)

[[0.8176060797973401, 0.8125395820139328, 0.811906269791007], [0.8188727042431919, 0.8277390753641545, 0.8220392653578215], [0.8283723875870804, 0.8030398986700443, 0.7973400886637112]]


In [11]:
ac=[]
sc=[]
i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    model_glove = Sequential()
    model_glove.add(Embedding(vocabulary_size, 300, input_length=80, weights=[embedding_matrix], trainable=False))
    model_glove.add(LSTM(300))
    model_glove.add(Dense(100, activation='relu'))
    model_glove.add(Dense(1, activation='sigmoid'))
    model_glove.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])
    
    print("Fold Number: "+str(i))  
    model_glove.fit(Xtrain, np.array(ytrain), epochs = 5)
    
    pred=model_glove.predict_classes(Xtest)
    
    a=accuracy_score(ytest, pred)
    s=f1_score(ytest, pred, average='macro')
    
    if i==0:
        # serialize model to JSON
        model_json = model_glove.to_json()
        with open("Models/model_Waseem_Hovy.json", "w") as json_file:
            json_file.write(model_json)
        # serialize weights to HDF5
        model_glove.save_weights("Models/model_Waseem_Hovy.h5")
        print("Saved model to disk")
    elif s>max(sc):
        # serialize model to JSON
        model_json = model_glove.to_json()
        with open("Models/model_Waseem_Hovy.json", "w") as json_file:
            json_file.write(model_json)
        # serialize weights to HDF5
        model_glove.save_weights("Models/model_Waseem_Hovy.h5")
        print("Saved model to disk")

    ac.append(a)
    sc.append(s)
    i=i+1    

Fold Number: 0
Epoch 1/5
11367/11367 [==============================] - 71s 6ms/step - loss: 0.4647 - acc: 0.7970
Epoch 2/5
11367/11367 [==============================] - 63s 6ms/step - loss: 0.3999 - acc: 0.8294
Epoch 3/5
11367/11367 [==============================] - 72s 6ms/step - loss: 0.3483 - acc: 0.8485
Epoch 4/5
11367/11367 [==============================] - 64s 6ms/step - loss: 0.2850 - acc: 0.8776
Epoch 5/5
11367/11367 [==============================] - 61s 5ms/step - loss: 0.2096 - acc: 0.9156
Saved model to disk
Fold Number: 1
Epoch 1/5
11368/11368 [==============================] - 61s 5ms/step - loss: 0.4611 - acc: 0.7942
Epoch 2/5
11368/11368 [==============================] - 60s 5ms/step - loss: 0.3970 - acc: 0.8269
Epoch 3/5
11368/11368 [==============================] - 69s 6ms/step - loss: 0.3447 - acc: 0.8493
Epoch 4/5
11368/11368 [==============================] - 79s 7ms/step - loss: 0.2848 - acc: 0.8799
Epoch 5/5
11368/11368 [==============================] - 74

In [12]:
print("Accuracies: ")
print(ac)
print("Mean: "+str(np.mean(np.array(ac))))
print("Std Dev: "+str(np.std(np.array(ac))))

Accuracies: 
[0.809004572634541, 0.814567206192822, 0.8152709359605911, 0.8268824771287826, 0.8240056318197818]
Mean: 0.8179461647473036
Std Dev: 0.00655898009856718


In [13]:
print("Macro-F1-Scores: ")
print(sc)
print("Mean: "+str(np.mean(np.array(sc))))
print("Std Dev: "+str(np.std(np.array(sc))))

Macro-F1-Scores: 
[0.780276307614792, 0.7740890877566365, 0.7856162213538916, 0.7885539457638929, 0.7825055120039197]
Mean: 0.7822082148986266
Std Dev: 0.004932011457477978


In [5]:
# load json and create model
json_file = open('Models/model_Waseem_Hovy.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_glove = model_from_json(loaded_model_json)
# load weights into new model
model_glove.load_weights("Models/model_Waseem_Hovy.h5")
print("Loaded model from disk")

Loaded model from disk


In [6]:
# Dataset-1
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Davidson.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

   Unnamed: 0                                               text  class
0           0   ! ! ! mayasolovely : woman complain cleaning ...      0
1           1   ! ! ! ! ! mleew17 : boy dats cold tyga dwn ba...      1
2           2   ! ! ! ! ! ! ! urkindofbrand dawg ! ! ! ! 80sb...      1
3           3   ! ! ! ! ! ! ! ! ! c g anderson : viva based l...      1
4           4   ! ! ! ! ! ! ! ! ! ! ! ! ! shenikaroberts : sh...      1


In [7]:
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_Waseem_Hovy.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    Waseem_Hovy_model = model_from_json(loaded_model_json)
    # load weights into new model
    Waseem_Hovy_model.load_weights("Models/model_Waseem_Hovy.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in Waseem_Hovy_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in Waseem_Hovy_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in Waseem_Hovy_model.layers:
        print(layer, layer.trainable)
    Waseem_Hovy_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    Waseem_Hovy_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=Waseem_Hovy_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7fc016a13c50> False
<keras.layers.recurrent.LSTM object at 0x7fc016a13d68> True
<keras.layers.core.Dense object at 0x7fc01681a4a8> True
<keras.layers.core.Dense object at 0x7fc013b51eb8> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7fc016a13c50> False
<keras.layers.recurrent.LSTM object at 0x7fc016a13d68> False
<keras.layers.core.Dense object at 0x7fc01681a4a8> True
<keras.layers.core.Dense object at 0x7fc013b51eb8> True
Epoch 1/8
4460/4460 [==============================] - 9s 2ms/step - loss: 0.4188 - acc: 0.8114
Epoch 2/8
4460/4460 [==============================] - 9s 2ms/step - loss: 0.2467 - acc: 0.8827
Epoch 3/8
4460/4460 [==============================] - 9s 2ms/step - loss: 0.2200 - acc: 0.8991
Epoch 4/8
4460/4460 [==============================] - 8s 2ms/step - loss: 0.2015 - acc: 0.9087
Epoch 5/8
4460/4460 [==============================] - 8s 2ms/step - loss: 

In [8]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))

With Fine Tuning: 
Accuracies: 
[0.9018377409233528, 0.9043030031376065, 0.9002242152466368, 0.9047085201793722, 0.9015695067264574]
Mean: 0.9025285972426851
Std Dev: 0.001709262633173571
Macro-F1-Scores: 
[0.8167993457722104, 0.8330496461731063, 0.8180775062969823, 0.8249038218077924, 0.8168867911382087]
Mean: 0.82194342223766
Std Dev: 0.006310127138779963


In [9]:
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

Without Fine Tuning: 
Accuracies: 
[0.525997310623039, 0.5291349170775437, 0.5369955156950673, 0.5228699551569507, 0.5334080717488789]
Mean: 0.5296811540602959
Std Dev: 0.00505274795322639
Macro-F1-Scores: 
[0.5022300703435166, 0.5054231580568945, 0.5135571406757847, 0.4986646382691007, 0.5085533727803382]
Mean: 0.5056856760251269
Std Dev: 0.005128132952807929


In [10]:
# Dataset-2
df=pd.read_csv('../Proper Experiments/Parsed Datasets/OffensEval_2019.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

   Unnamed: 0                                               text  class
0           0                 user ask native americans take is       1
1           1   user user home you re drunk ! ! ! user maga t...      1
2           2  amazon investigating chinese employees selling...      0
3           3   user someone should have taken piece shit vol...      1
4           4   user user obama wanted liberals amp; illegals...      0


In [11]:
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_Waseem_Hovy.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    Waseem_Hovy_model = model_from_json(loaded_model_json)
    # load weights into new model
    Waseem_Hovy_model.load_weights("Models/model_Waseem_Hovy.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in Waseem_Hovy_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in Waseem_Hovy_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in Waseem_Hovy_model.layers:
        print(layer, layer.trainable)
    Waseem_Hovy_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    Waseem_Hovy_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=Waseem_Hovy_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7fbf962020b8> False
<keras.layers.recurrent.LSTM object at 0x7fbf96202748> True
<keras.layers.core.Dense object at 0x7fbf96202390> True
<keras.layers.core.Dense object at 0x7fc007e66cc0> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7fbf962020b8> False
<keras.layers.recurrent.LSTM object at 0x7fbf96202748> False
<keras.layers.core.Dense object at 0x7fbf96202390> True
<keras.layers.core.Dense object at 0x7fc007e66cc0> True
Epoch 1/8
2383/2383 [==============================] - 4s 2ms/step - loss: 0.8665 - acc: 0.6597
Epoch 2/8
2383/2383 [==============================] - 4s 2ms/step - loss: 0.5857 - acc: 0.6979
Epoch 3/8
2383/2383 [==============================] - 4s 2ms/step - loss: 0.5537 - acc: 0.7159
Epoch 4/8
2383/2383 [==============================] - 4s 2ms/step - loss: 0.5338 - acc: 0.7277
Epoch 5/8
2383/2383 [==============================] - 4s 2ms/step - loss: 

In [12]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))

With Fine Tuning: 
Accuracies: 
[0.7399328859060402, 0.7192614351657575, 0.7259756609315988, 0.7213596307175829, 0.7322702475870752]
Mean: 0.7277599720616109
Std Dev: 0.007548452578885848
Macro-F1-Scores: 
[0.6727780217911843, 0.6460131187421354, 0.6659720357387182, 0.6616597044746195, 0.6682246294582845]
Mean: 0.6629295020409883
Std Dev: 0.009187656248581463


In [13]:
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

Without Fine Tuning: 
Accuracies: 
[0.6715604026845637, 0.6693243810323122, 0.6697440201426773, 0.6701636592530423, 0.6651279899286614]
Mean: 0.6691840906082513
Std Dev: 0.0021629410081396776
Macro-F1-Scores: 
[0.47745158719427855, 0.4778714506567606, 0.4772743426075173, 0.4697644477056241, 0.4643296578486584]
Mean: 0.47333829720256776
Std Dev: 0.0054201304681825236


In [14]:
# Dataset-3
df=pd.read_csv('../Proper Experiments/Parsed Datasets/hatEval_2019.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

   Unnamed: 0                                               text  class
0           0  hurray saving many ways potus realdonaldtrump ...      0
1           1  would young fighting age men vast majority one...      0
2           2   kamalaharris illegals dump kids border like r...      0
3           3  times : nearly white states pose an array prob...      0
4           4  orban brussels : european leaders ignoring peo...      0


In [15]:
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_Waseem_Hovy.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    Waseem_Hovy_model = model_from_json(loaded_model_json)
    # load weights into new model
    Waseem_Hovy_model.load_weights("Models/model_Waseem_Hovy.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in Waseem_Hovy_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in Waseem_Hovy_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in Waseem_Hovy_model.layers:
        print(layer, layer.trainable)
    Waseem_Hovy_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    Waseem_Hovy_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=Waseem_Hovy_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7fbf887fba20> False
<keras.layers.recurrent.LSTM object at 0x7fbf887fb5c0> True
<keras.layers.core.Dense object at 0x7fbf887fb4a8> True
<keras.layers.core.Dense object at 0x7fbf887fbe80> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7fbf887fba20> False
<keras.layers.recurrent.LSTM object at 0x7fbf887fb5c0> False
<keras.layers.core.Dense object at 0x7fbf887fb4a8> True
<keras.layers.core.Dense object at 0x7fbf887fbe80> True
Epoch 1/8
1618/1618 [==============================] - 3s 2ms/step - loss: 0.6827 - acc: 0.7386
Epoch 2/8
1618/1618 [==============================] - 3s 2ms/step - loss: 0.4626 - acc: 0.8208
Epoch 3/8
1618/1618 [==============================] - 3s 2ms/step - loss: 0.4212 - acc: 0.8269
Epoch 4/8
1618/1618 [==============================] - 3s 2ms/step - loss: 0.4029 - acc: 0.8337
Epoch 5/8
1618/1618 [==============================] - 2s 2ms/step - loss: 

In [16]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))

With Fine Tuning: 
Accuracies: 
[0.830142063001853, 0.8213844252163165, 0.8238566131025958, 0.8145859085290482, 0.8176761433868974]
Mean: 0.8215290306473422
Std Dev: 0.005341807973856368
Macro-F1-Scores: 
[0.5372925234120786, 0.5188463062145323, 0.5229849208491992, 0.5242239969889084, 0.5356016668563917]
Mean: 0.527789882864222
Std Dev: 0.007309102706249691


In [17]:
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

Without Fine Tuning: 
Accuracies: 
[0.6454601605929586, 0.6291718170580964, 0.6489493201483313, 0.6390605686032138, 0.6372064276885043]
Mean: 0.6399696588182209
Std Dev: 0.006867826516647347
Macro-F1-Scores: 
[0.5169766561881476, 0.5138659375657242, 0.5315461529678503, 0.5319548574236396, 0.5299107352770127]
Mean: 0.5248508678844749
Std Dev: 0.007791889009690587


In [18]:
# Dataset-4
df=pd.read_csv('../Proper Experiments/Parsed Datasets/StormfrontWS.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

   Unnamed: 0  class                                               text
0           0      0  march 13th 2014 booklet downloaded 18 300 time...
1           1      0  order help increase booklets downloads would g...
2           2      0  simply copy paste following text youtube video...
3           3      1  click free download colorfully illustrated 132...
4           4      0              click download 7 42 green banner link


In [19]:
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_Waseem_Hovy.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    Waseem_Hovy_model = model_from_json(loaded_model_json)
    # load weights into new model
    Waseem_Hovy_model.load_weights("Models/model_Waseem_Hovy.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in Waseem_Hovy_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in Waseem_Hovy_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in Waseem_Hovy_model.layers:
        print(layer, layer.trainable)
    Waseem_Hovy_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    Waseem_Hovy_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=Waseem_Hovy_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7fbf8e5af8d0> False
<keras.layers.recurrent.LSTM object at 0x7fbf8678c198> True
<keras.layers.core.Dense object at 0x7fbf8a904dd8> True
<keras.layers.core.Dense object at 0x7fbf865bffd0> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7fbf8e5af8d0> False
<keras.layers.recurrent.LSTM object at 0x7fbf8678c198> False
<keras.layers.core.Dense object at 0x7fbf8a904dd8> True
<keras.layers.core.Dense object at 0x7fbf865bffd0> True
Epoch 1/8
1926/1926 [==============================] - 4s 2ms/step - loss: 0.3966 - acc: 0.8583
Epoch 2/8
1926/1926 [==============================] - 3s 2ms/step - loss: 0.3209 - acc: 0.8853
Epoch 3/8
1926/1926 [==============================] - 3s 2ms/step - loss: 0.2957 - acc: 0.8936
Epoch 4/8
1926/1926 [==============================] - 3s 2ms/step - loss: 0.2778 - acc: 0.8982
Epoch 5/8
1926/1926 [==============================] - 4s 2ms/step - loss: 

In [20]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))

With Fine Tuning: 
Accuracies: 
[0.8874481327800829, 0.8811007268951194, 0.8826583592938733, 0.8888888888888888, 0.8940809968847352]
Mean: 0.88683542094854
Std Dev: 0.004635193134507959
Macro-F1-Scores: 
[0.5476737485445049, 0.5485939040004422, 0.5915202702702702, 0.5879019356902896, 0.6172138047138047]
Mean: 0.5785807326438623
Std Dev: 0.026837594064994315


In [21]:
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

Without Fine Tuning: 
Accuracies: 
[0.8412863070539419, 0.8333333333333334, 0.842159916926272, 0.8312564901349948, 0.8385254413291796]
Mean: 0.8373122977555443
Std Dev: 0.00431900087216328
Macro-F1-Scores: 
[0.5806264501160093, 0.5789854317461679, 0.5903778991671891, 0.5903326489727719, 0.5970209802912416]
Mean: 0.587468682058676
Std Dev: 0.0067334081417257715


In [22]:
# Dataset-5
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Davidson_1.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

   Unnamed: 0                                               text  class
0           0   ! ! ! mayasolovely : woman complain cleaning ...      0
1           1   ! ! ! ! ! mleew17 : boy dats cold tyga dwn ba...      0
2           2   ! ! ! ! ! ! ! urkindofbrand dawg ! ! ! ! 80sb...      0
3           3   ! ! ! ! ! ! ! ! ! c g anderson : viva based l...      0
4           4   ! ! ! ! ! ! ! ! ! ! ! ! ! shenikaroberts : sh...      0


In [23]:
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_Waseem_Hovy.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    Waseem_Hovy_model = model_from_json(loaded_model_json)
    # load weights into new model
    Waseem_Hovy_model.load_weights("Models/model_Waseem_Hovy.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in Waseem_Hovy_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in Waseem_Hovy_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in Waseem_Hovy_model.layers:
        print(layer, layer.trainable)
    Waseem_Hovy_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    Waseem_Hovy_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=Waseem_Hovy_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7fbf6c776e10> False
<keras.layers.recurrent.LSTM object at 0x7fbf6c776f98> True
<keras.layers.core.Dense object at 0x7fbf6c772390> True
<keras.layers.core.Dense object at 0x7fbf6a2eada0> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7fbf6c776e10> False
<keras.layers.recurrent.LSTM object at 0x7fbf6c776f98> False
<keras.layers.core.Dense object at 0x7fbf6c772390> True
<keras.layers.core.Dense object at 0x7fbf6a2eada0> True
Epoch 1/8
4460/4460 [==============================] - 10s 2ms/step - loss: 0.3312 - acc: 0.9101
Epoch 2/8
4460/4460 [==============================] - 7s 2ms/step - loss: 0.2085 - acc: 0.9424
Epoch 3/8
4460/4460 [==============================] - 9s 2ms/step - loss: 0.1860 - acc: 0.9433
Epoch 4/8
4460/4460 [==============================] - 9s 2ms/step - loss: 0.1750 - acc: 0.9426
Epoch 5/8
4460/4460 [==============================] - 7s 2ms/step - loss:

In [24]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))

With Fine Tuning: 
Accuracies: 
[0.942626624831914, 0.9430748543254146, 0.9412556053811659, 0.9426008968609866, 0.9374439461883408]
Mean: 0.9414003855175643
Std Dev: 0.0020699328290515995
Macro-F1-Scores: 
[0.5613131969503549, 0.6049758457473877, 0.5481361175560712, 0.5527252670109813, 0.5518848644371949]
Mean: 0.563807058340398
Std Dev: 0.02103146020986108


In [25]:
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

Without Fine Tuning: 
Accuracies: 
[0.5658897355445989, 0.5999551770506499, 0.5840807174887892, 0.5834080717488789, 0.5957399103139014]
Mean: 0.5858147224293637
Std Dev: 0.011868156170441086
Macro-F1-Scores: 
[0.3889634400811671, 0.4119373903335926, 0.39857100185074645, 0.40210094559290904, 0.4145864817873364]
Mean: 0.40323185192915034
Std Dev: 0.0092876699702994


In [6]:
# Dataset-6
df=pd.read_csv('../Proper Experiments/Parsed Datasets/GAB.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_Waseem_Hovy.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    Waseem_Hovy_model = model_from_json(loaded_model_json)
    # load weights into new model
    Waseem_Hovy_model.load_weights("Models/model_Waseem_Hovy.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in Waseem_Hovy_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in Waseem_Hovy_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in Waseem_Hovy_model.layers:
        print(layer, layer.trainable)
    Waseem_Hovy_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    Waseem_Hovy_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=Waseem_Hovy_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0  class                                               text
0           0      1  joined gab remind retarded jew haters are typi...
1           1      0       left really scared of https : redd it 9rfkts
2           2      0   that literally looks like monkey supposed pre...
3           3      1                                          dumb cunt
4           4      0                                     makes asshole 
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f80404a7ef0> False
<keras.layers.recurrent.LSTM object at 0x7f80404a7438> True
<keras.layers.core.Dense object at 0x7f804f3a3b00> True
<keras.layers.core.Dense object at 0x7f804c70a748> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f80404a7ef0> False
<keras.layers.recurrent.LSTM object at 0x7f80404a7438> False
<keras.layers.core.Dense object at 0x7f804f3a3b00> True
<keras.layers.core.Dense object at 0x7f804c70a748> True
Epoch 1/8
6079/6079 [======

In [7]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.8015129090610097, 0.8036184210526316, 0.8093436420463892, 0.7879585458134561, 0.7922355650600428]
Mean: 0.7989338166067059
Std Dev: 0.007775407965564857
Macro-F1-Scores: 
[0.7949239323852241, 0.7965147568510107, 0.8024195272821203, 0.7825583505291043, 0.7853600692212915]
Mean: 0.7923553272537502
Std Dev: 0.007349844855382685
Without Fine Tuning: 
Accuracies: 
[0.6378885051800691, 0.6271381578947368, 0.6359598618193781, 0.6341503536765916, 0.6402368810659648]
Mean: 0.6350747519273481
Std Dev: 0.004454037294329845
Macro-F1-Scores: 
[0.5986675162656647, 0.5873008145169067, 0.5933394167451578, 0.5981719981406121, 0.6014687603278533]
Mean: 0.5957897011992389
Std Dev: 0.004985458090709626


In [8]:
# Dataset-7
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Reddit.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_Waseem_Hovy.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    Waseem_Hovy_model = model_from_json(loaded_model_json)
    # load weights into new model
    Waseem_Hovy_model.load_weights("Models/model_Waseem_Hovy.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in Waseem_Hovy_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in Waseem_Hovy_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in Waseem_Hovy_model.layers:
        print(layer, layer.trainable)
    Waseem_Hovy_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    Waseem_Hovy_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=Waseem_Hovy_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0  class                                               text
0           0      1  subsection retarded hungarians ohh boy brace l...
1           1      0   hiii got work 444 mainly typa guys imagine wr...
2           2      0                    wow guess soyboys every country
3           3      0   owen benjamin soyboy song goes every country ...
4           4      0   y all hear sumn means live small town for wor...
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f80404ccdd8> False
<keras.layers.recurrent.LSTM object at 0x7f80404cc550> True
<keras.layers.core.Dense object at 0x7f80404ccd30> True
<keras.layers.core.Dense object at 0x7f80406464a8> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f80404ccdd8> False
<keras.layers.recurrent.LSTM object at 0x7f80404cc550> False
<keras.layers.core.Dense object at 0x7f80404ccd30> True
<keras.layers.core.Dense object at 0x7f80406464a8> True
Epoch 1/8
4018/4018 [======

In [9]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.8355312266733018, 0.827775012444002, 0.8337481333997013, 0.8208063713290195, 0.8325037332005973]
Mean: 0.8300728954093243
Std Dev: 0.0052982725768668075
Macro-F1-Scores: 
[0.7322650038876899, 0.7324920722884148, 0.7306313206735201, 0.7264269024893699, 0.7248884568408983]
Mean: 0.7293407512359786
Std Dev: 0.003113232638858231
Without Fine Tuning: 
Accuracies: 
[0.6989300821099776, 0.7090592334494773, 0.7073170731707317, 0.7033349925335988, 0.7075659532105525]
Mean: 0.7052414668948676
Std Dev: 0.0036811667145845175
Macro-F1-Scores: 
[0.5689416454106506, 0.5800511319161415, 0.5766926763717805, 0.5743227941816915, 0.569936403439853]
Mean: 0.5739889302640233
Std Dev: 0.004148977104139846


In [6]:
# Dataset-8
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Davidson_2.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_Waseem_Hovy.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    Waseem_Hovy_model = model_from_json(loaded_model_json)
    # load weights into new model
    Waseem_Hovy_model.load_weights("Models/model_Waseem_Hovy.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in Waseem_Hovy_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in Waseem_Hovy_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in Waseem_Hovy_model.layers:
        print(layer, layer.trainable)
    Waseem_Hovy_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    Waseem_Hovy_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=Waseem_Hovy_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0  class                                               text
0           0      0   ! ! ! mayasolovely : woman complain cleaning ...
1           1      0              momma said pussy cats inside doghouse
2           2      0   addicted2guys : - simplyaddictedtoguys http :...
3           3      0   allaboutmanfeet : http : t co 3gzupfumev woof...
4           4      0   allyhaaaaa : lemmie eat oreo amp; dishes one ...
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f5af40906a0> False
<keras.layers.recurrent.LSTM object at 0x7f5af4090a20> True
<keras.layers.core.Dense object at 0x7f5af4090978> True
<keras.layers.core.Dense object at 0x7f5af92c4630> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f5af40906a0> False
<keras.layers.recurrent.LSTM object at 0x7f5af4090a20> False
<keras.layers.core.Dense object at 0x7f5af4090978> True
<keras.layers.core.Dense object at 0x7f5af92c4630> True
Epoch 1/8
1006/1006 [======

In [7]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.8541666666666666, 0.8669314796425025, 0.8508946322067594, 0.8648111332007953, 0.8598409542743539]
Mean: 0.8593289731982157
Std Dev: 0.006097570398731992
Macro-F1-Scores: 
[0.7873597775095825, 0.8123776418242492, 0.7803928188888372, 0.8106594039236525, 0.8028150217903538]
Mean: 0.798720932787335
Std Dev: 0.01273412712263996
Without Fine Tuning: 
Accuracies: 
[0.7559523809523809, 0.7586891757696127, 0.7504970178926441, 0.7584493041749503, 0.768389662027833]
Mean: 0.7583955081634841
Std Dev: 0.00580219381382948
Macro-F1-Scores: 
[0.5995115995115996, 0.6098299490564684, 0.6105868509485263, 0.6081819886617716, 0.6385128935099864]
Mean: 0.6133246563376705
Std Dev: 0.013201540962205513
